**Scraping Laureados (selenium)**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Iniciamos el navegador Chrome para realizar web scraping dinámico.
driver = webdriver.Chrome()

# Accedemos a la página de Wikipedia que contiene la lista de laureados por país.
driver.get("https://es.wikipedia.org/wiki/Anexo:Laureados_de_los_Premios_Nobel_por_pa%C3%ADs")

# Cada país aparece como un encabezado <h3> con la clase 'mw-heading3'.
# A partir de cada encabezado, extraeremos la lista de laureados asociada.
paises = driver.find_elements(By.CLASS_NAME, 'mw-heading.mw-heading3')

# Creamos el archivo CSV donde guardaremos los datos extraídos.
with open("laureados.csv", "w", encoding="utf-8", newline='') as f:
    
    # Cabecera del archivo, usando ';' como separador.
    f.write('pais;nombre;categoria;anyo\n')

    # Recorremos cada país encontrado en la página.
    for pais in paises:

        # El texto del encabezado incluye "[editar]" al final, así que eliminamos los últimos 8 caracteres.
        pais_imp = pais.text[:-8]

        # Cada país va seguido de una lista <ol> con los laureados.
        # Usamos 'following-sibling::ol[1]' para obtener la primera lista que sigue al encabezado.
        div_pais = pais.find_element(By.XPATH, "./following-sibling::ol[1]")

        # Cada laureado está en un elemento <li> dentro de esa lista.
        filas = div_pais.find_elements(By.TAG_NAME, "li")

        # Procesamos cada laureado de la lista.
        for i in filas:

            # Sustituimos posibles guiones por comas para separar nombre, categoría y año.
            i = i.text.replace('-', ',')

            # Dividimos la línea en partes: [nombre, categoria, año]
            i = i.split(',')

            # Eliminamos espacios residuales en las partes posteriores al nombre.
            for j in i[1:]:
                j.replace(' ', '')

            # Algunos laureados tienen información adicional (por ejemplo, nacionalidades múltiples).
            # Si hay más de 3 elementos, nos quedamos solo con: nombre, categoría, año.
            if len(i) > 3:
                i = [i[0], i[-2], i[-1]]

            # Excluimos la categoría de la Paz, ya que no forma parte del análisis.
            if 'Paz' not in i[1]:

                # Construimos la línea final del CSV.
                texto = pais_imp + ';' + ';'.join(i)

                # Eliminamos posibles asteriscos de Wikipedia.
                texto = texto.replace('*', '')

                # Guardamos la línea en el archivo.
                f.write(texto + '\n')


**Scraping de gasto en educación (selenium y filtrado con re)**

In [2]:

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

# Iniciamos el navegador Chrome para realizar web scraping dinámico.
driver = webdriver.Chrome()

# Accedemos a la página de DatosMacro con el gasto en educación (% del PIB).
driver.get("https://datosmacro.expansion.com/estado/gasto/educacion")

# Cerramos el aviso de cookies para poder interactuar con la página.
driver.find_element(By.ID, 'ue-disagree-notice-button').click()

# Creamos el archivo CSV donde guardaremos los datos extraídos.
with open("gasto_educacion2.csv", "w", encoding="utf-8", newline='') as f:
    
    # Cabecera del archivo, usando ';' como separador.
    f.write('pais;anyo;gasto_pib\n')

    # La tabla está paginada: avanzamos página a página hasta que no queden más.
    while True:
        try:
            # Localizamos la tabla principal de datos.
            tabla = driver.find_element(By.ID, 'tb1')

            # Extraemos todas las filas (<tr>) de la tabla.
            texto = tabla.find_elements(By.TAG_NAME, 'tr')

            # La primera fila contiene los títulos, el resto son datos.
            cuerpo = texto[1:]

            # Recorremos cada fila de datos.
            for linea in cuerpo:
                try:
                    # Extraemos el nombre del país usando una expresión regular.
                    # Busca un texto que empiece en mayúscula y termine en minúscula.
                    nombre = re.findall(r'[A-ZÁÉÍÓÚ].+[a-záéíúó]', linea.text)

                    # Extraemos el año (columna con clase 'fecha').
                    anyo = linea.find_element(By.CLASS_NAME, 'fecha')

                    # Extraemos todas las columnas numéricas.
                    numeros = linea.find_elements(By.CLASS_NAME, 'numero')

                    # El gasto en educación (% PIB) está en la cuarta columna empezando por el final.
                    # Aclaración: En la página web es la tercera columna, pero al inspeccionar la página 
                    # vemos que hay una columna extra llamada "numero dol" (Gasto Educación Per Cápita en dólares).
                    gasto_PIB = numeros[-4]

                    # Guardamos la línea en el CSV.
                    f.write(f'{nombre[0]};{anyo.text};{gasto_PIB.text}\n')

                except:
                    # Algunas filas pueden no tener datos completos, las ignoramos.
                    pass

            # Pasamos a la página anterior (la web ordena los años de más reciente a más antiguo).
            driver.find_element(By.CLASS_NAME, 'float-start.padleft').click()

        except:
            # Si Selenium no encuentra la tabla o el botón de navegación,
            # significa que hemos llegado al final de las páginas.
            break

        

**Scraping de gasto en investigación (GET HTTP y normalización de nombres)**

In [3]:
import requests

# Diccionario para traducir los nombres de países del inglés (OWID) al español, garantizando coherencia con el dataset de laureados.
paises = {"Germany": "Alemania", 
          "Argentina": "Argentina",
          "Australia": "Australia",
          "Austria": "Austria",
          "Azerbaijan":"Azerbaiyán",
          "Belgium": "Bélgica",
          "Belarus": "Bielorrusia",
          "Bosnia and Herzegovina": "Bosnia y Herzegovina",
          "Bulgaria": "Bulgaria",
          "Canada": "Canadá",
          "Czechia":"República Checa",
          "Chile":"Chile",
          "China":"República Popular China",
          "Cyprus":"Chipre",
          "Colombia":"Colombia",
          "South Korea":"Corea del Sur", 
          "Croatia":"Croacia",
          "Denmark":"Dinamarca", 
          "Egypt":"Egipto",
          "Slovenia":"Eslovenia",
          "Spain":"España",
          "United States": "Estados Unidos", 
          "Faroe Islands":"Islas Feroe",
          "Finland":"Finlandia",
          "France":"Francia", 
          "Greece":"Grecia",
          "Guatemala":"Guatemala", 
          "Hong Kong":"Hong Kong",
          "Hungary":"Hungría",
          "India":"India",
          "Ireland":"Irlanda",
          "Israel":"Israel",
          "Italy":"Italia",
          "Iceland":"Islandia",
          "Japan":"Japón",
          "Lithuania":"Lituania",
          "Luxembourg":"Luxemburgo",
          "Mexico": "México",
          "Nigeria":"Nigeria",
          "Norway":"Noruega", 
          "New Zealand":"Nueva Zelanda",
          "Netherlands":"Países Bajos",
          "Pakistan":"Pakistán",
          "Peru":"Perú",
          "Poland":"Polonia",
          "Portugal":"Portugal",
          "United Kingdom":"Reino Unido",
          "Romania":"Rumania",
          "Russia":"Rusia y Unión Soviética",
          "Saint Lucia": "Santa Lucía",
          "South Africa":"Sudáfrica",
          "Sweden":"Suecia",
          "Switzerland":"Suiza",
          "Trinidad and Tobago": "Trinidad y Tobago",
          "Turkey":"Turquía",
          "Ukraine":"Ucrania",
          "Venezuela":"Venezuela"}

# Descargamos el CSV de Our World in Data mediante una petición HTTP.
# Este archivo contiene el gasto en I+D (% del PIB) por país y año.
url = "https://ourworldindata.org/grapher/research-spending-gdp.csv"
texto = requests.get(url).text

# Convertimos el CSV en una lista de líneas para procesarlo manualmente.
l = texto.split("\n") 


# Creamos un nuevo CSV filtrado y traducido, solo con los países que aparecen en el dataset de laureados.
with open("research_spending.csv", "w", encoding="utf-8") as f:

    # Cabecera del nuevo archivo, usando ';' como separador.
    f.write("pais;codigo;anyo;investigacion_pib\n")

    # Recorremos cada línea del CSV original.
    for linea in l:
        # Comprobamos si la línea pertenece a un país relevante.
        for en, es in paises.items():
            if linea.startswith(en + ","):

                # Sustituimos el nombre del país por su traducción al español.
                linea = linea.replace(en, es, 1)

                # Cambiamos el separador ',' por ';'.
                linea = linea.replace(",", ";") 
                
                # Escribimos la línea traducida y filtrada en el nuevo archivo.
                f.write(linea + "\n")
